# Eager Execution で遊ぶ
いろいろ試してみる

In [2]:
import tensorflow as tf

In [3]:
tf.enable_eager_execution()

# 単純計算

In [ ]:
x = tf.constant(1.)
y = tf.constant(2.)
x

In [ ]:
for i in range(10):
    z = y
    y = x + y
    x = z
    print(y)

# パフォーマンス

In [4]:
import time
from tqdm import tqdm

In [5]:
def calc():
    X = tf.ones(shape=[256, 256], dtype=tf.float32)
    for i in range(100):
        X = tf.layers.dense(X, 256)
    return X

In [6]:
start = time.time()
with tf.Graph().as_default():
    X = calc()
    after_graph = time.time()
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        result = sess.run(X)
end = time.time()
print('graph mode elapsed:', end - start)
print('graph mode run:', end - after_graph)
#print(result)

graph mode elapsed: 1.8623850345611572
graph mode run: 1.0190224647521973


In [7]:
start = time.time()
result = calc()
print('eager mode elapsed:', time.time() - start)
#print(result)

eager mode elapsed: 0.23740339279174805


In [8]:
start = time.time()
elapsed_list = []
with tf.Graph().as_default():
    X = calc()
    with tf.Session() as sess:
        for i in tqdm(range(50)):
            iter_start = time.time()
            sess.run(tf.global_variables_initializer())
            result = sess.run(X)
            elapsed_list.append(time.time() - iter_start)

print('graph mode elapsed:', time.time() - start)
print(' '.join([f'{elapsed:.3f}' for elapsed in elapsed_list]))
#print(result)

100%|██████████| 50/50 [00:12<00:00,  3.95it/s]

graph mode elapsed: 13.353304862976074
0.336 0.236 0.237 0.238 0.238 0.239 0.234 0.231 0.237 0.236 0.238 0.239 0.241 0.242 0.239 0.240 0.240 0.239 0.241 0.241 0.245 0.246 0.242 0.244 0.252 0.248 0.248 0.246 0.248 0.249 0.247 0.248 0.248 0.247 0.248 0.249 0.250 0.250 0.249 0.249 0.250 0.249 0.252 0.251 0.252 0.252 0.252 0.254 0.253 0.253


In [9]:
start = time.time()
elapsed_list = []
result = calc()

for i in tqdm(range(50)):
    iter_start = time.time()
    result = calc()
    elapsed_list.append(time.time() - iter_start)
print('eager mode elapsed:', time.time() - start)
print(' '.join([f'{elapsed:.3f}' for elapsed in elapsed_list]))
#print(result)

100%|██████████| 50/50 [00:21<00:00,  1.74it/s]

eager mode elapsed: 21.258570671081543
0.250 0.258 0.264 0.272 0.278 0.284 0.291 0.297 0.304 0.316 0.327 0.336 0.342 0.348 0.355 0.363 0.369 0.375 0.374 0.377 0.395 0.402 0.408 0.407 0.414 0.419 0.425 0.432 0.439 0.445 0.453 0.457 0.466 0.474 0.481 0.488 0.495 0.502 0.513 0.519 0.525 0.534 0.540 0.547 0.554 0.562 0.569 0.576 0.583 0.589


In [ ]:
g = tf.get_default_graph()
g.get_operations()

# パフォーマンス（Model）

In [10]:
import time
from tqdm import tqdm

In [11]:
class Calc(tf.keras.Model):
    def __init__(self):
        super(Calc, self).__init__()
        self.denses = [tf.keras.layers.Dense(units=256) for i in range(100)]
        
    def call(self, input):
        x = input
        for layer in self.denses:
            x = layer(x)
        return x

In [15]:
start = time.time()
elapsed_list = []
#calc_model = Calc()

for i in tqdm(range(50)):
    iter_start = time.time()
    input_ = tf.ones(shape=[256, 256], dtype=tf.float32)
#    result = calc_model(input_)
    result = Calc()(input_)
    elapsed_list.append(time.time() - iter_start)
print('eager mode elapsed:', time.time() - start)
print(' '.join([f'{elapsed:.3f}' for elapsed in elapsed_list]))
print(result)

100%|██████████| 50/50 [00:10<00:00,  4.60it/s]

eager mode elapsed: 10.831282377243042
0.212 0.213 0.213 0.214 0.213 0.214 0.216 0.216 0.214 0.215 0.259 0.222 0.215 0.218 0.217 0.217 0.215 0.210 0.215 0.215 0.211 0.210 0.214 0.214 0.214 0.214 0.212 0.215 0.214 0.214 0.209 0.215 0.219 0.215 0.211 0.214 0.214 0.214 0.212 0.215 0.212 0.214 0.215 0.262 0.221 0.215 0.215 0.215 0.215 0.215
tf.Tensor(
[[ 0.79088724 -0.37703872 -0.20463532 ...  0.13335949 -0.7566216
  -0.03036916]
 [ 0.79088724 -0.37703872 -0.20463532 ...  0.13335949 -0.7566216
  -0.03036916]
 [ 0.79088724 -0.37703872 -0.20463532 ...  0.13335949 -0.7566216
  -0.03036916]
 ...
 [ 0.79088724 -0.37703872 -0.20463532 ...  0.13335949 -0.7566216
  -0.03036916]
 [ 0.79088724 -0.37703872 -0.20463532 ...  0.13335949 -0.7566216
  -0.03036916]
 [ 0.79088724 -0.37703872 -0.20463532 ...  0.13335949 -0.7566216
  -0.03036916]], shape=(256, 256), dtype=float32)


# autograph

In [ ]:
import numpy as np
import random
from tensorflow.contrib import autograph

In [ ]:
def step_generator(input_: int):
    if input_ <= 0 or input_ > 10:
        return 0
    return input_ - 1
#    return input_ + np.random.choice([-1, 1])  # this occures ERROR on to_graph()
#    return input_ + random.randint(-1, 1)

def loop_generator(i: int):
    i_list = [i]
#    while i != 0:  # this occures ERROR on run graph on graph-mode after to_graph()
    for _ in range(10):
#        if i == 0: # this occures ERROR on run graph on graph-mode after to_graph()
#            break
        i = step_generator(i)
        i_list.append(i)
    return i_list

In [ ]:
result = loop_generator(tf.constant(1))
print('\n'.join(['{}: {}'.format(i, type(i)) for i in result]))


In [ ]:
tf_step_generator = autograph.to_graph(step_generator)
tf_loop_generator = autograph.to_graph(loop_generator)

In [ ]:
print(autograph.to_code(loop_generator))

In [ ]:
tf_loop_generator(tf.constant(1))

In [ ]:
with tf.Session(graph=tf.Graph()) as sess:
    input_op = tf.constant(1)
    result = sess.run(tf_loop_generator(input_op))
    print(result)